# Mike Babb
# babbm@uw.edu
# Find anagrams
## Part 2: Generate and store the anagrams v2.0

In [1]:
# standard libraries - installed by default
import collections
import datetime
from itertools import product
import pickle
import sqlite3
import string
import os
import timeit

In [2]:
# external libraries - not installed by default
import numpy as np
import pandas as pd

In [3]:
from part_00_file_db_utils import *
from part_00_process_functions import *

### set input and output paths

In [4]:
# base file path
base_file_path = '/project/finding_anagrams'

In [5]:
# input path
in_file_path = 'data'
in_file_path = os.path.join(base_file_path, in_file_path)

In [6]:
# output db path and name
db_path = 'db'
db_path = os.path.join(base_file_path, db_path)

In [7]:
if os.path.exists(db_path):
    pass
else:
    os.makedirs(db_path)

In [8]:
db_name = 'words.db'

### process control flags

In [9]:
# Use numpy to perform matrix opertions and determine from/to and exact anagram relationships
# Option 1: Full matrix
# Option 2: Word-length
# Option 3: First letter
# Option 4: Single-least common letter
# Option 5: n least common letters
# Option 6: word-length and n least common letters

matrix_extraction_option = 6

# max number of letters to slice to use for the generation of sub-matrices for
# options 5 and 6. More letters means more sub-matrices
# 3 seems to be the sweet spot
n_subset_letters = 3

# set write_data to True to store the generated list of anagrams
write_data = False

# set to None to include all letters
# test with a subset of letters by setting the letter_subset_list to ['q', 'x'] or 
# a different set of letters
letter_subset_list = ['x']
# letter_subset_list = None

# generate a sample dataset of ten words that start with each letter

# testo = wg_df.groupby(['first_letter']).sample(n = 10, random_state = 123)


In [10]:
# start a timer to record the entire operation
total_time_start = datetime.datetime.now()

### load input data

In [11]:
word_df, wg_df, letter_dict, char_matrix, word_group_id_list, word_id_list, wchar_matrix = load_input_data(db_path = db_path, db_name = db_name, in_file_path = in_file_path)

...loading words into a dataframe...
...query execution took: 1.76776 seconds...
...loading word groups into a dataframe...
...query execution took: 1.727285 seconds...
...loading the letter dictionary...
...loading the char matrix...
...subsetting the char matrix...


In [12]:
# Split the char_matrix into N sub matrices
# See split_matrix() for a more elaborate description. 
# This function does a lot of things. Effectively, it computes and stores values in the wg_df, and splits the matrix into various components.

In [13]:
wg_df, n_char_matrix_dict, single_letter_matrix_dict, letter_selector_matrix_dict, nc_ls_matrix_dict= split_matrix(
    letter_dict = letter_dict,
    word_group_id_list = word_group_id_list,
        wg_df = wg_df,
    wchar_matrix = wchar_matrix, 
    n_subset_letters = n_subset_letters,
)

...creating 16,101 sub-matrices...
...1,000 sub-matrices created...
...2,000 sub-matrices created...
...3,000 sub-matrices created...
...4,000 sub-matrices created...
...5,000 sub-matrices created...
...6,000 sub-matrices created...
...7,000 sub-matrices created...
...8,000 sub-matrices created...
...9,000 sub-matrices created...
...10,000 sub-matrices created...
...11,000 sub-matrices created...
...12,000 sub-matrices created...
...13,000 sub-matrices created...
...14,000 sub-matrices created...
...15,000 sub-matrices created...
...16,000 sub-matrices created...
... 16,101 sub-matrices created...
Total extraction time: 59.39 seconds.


In [ ]:
test_matrix_extraction_option(wg_df = wg_df,
                                  letter_dict = letter_dict,
                                  word_group_id_list = word_group_id_list,
                                  wchar_matrix = wchar_matrix,
                                  n_subset_letters = n_subset_letter):

In [ ]:
wg_lu_df = compute_lookups(wg_df=wg_df,
                    n_char_matrix_dict = n_char_matrix_dict,
                    single_letter_matrix_dict = single_letter_matrix_dict,
                    letter_selector_matrix_dict = letter_selector_matrix_dict,
                    nc_ls_matrix_dict = nc_ls_matrix_dict,
                          db_path = db_path, db_name = db_name)

# demonstrate the different matrix extraction options with the word 'achiever'

In [14]:
demo_word = 'achiever'

wg_id = word_df.loc[word_df['lcase'] == demo_word, 'word_group_id'].iloc[0]

demo_wg_df = wg_df.loc[wg_df['word_group_id'] == wg_id, : ]

# option 1 - Full matrix
# No additional data needed

# option 2 -  Number of characters
n_char = demo_wg_df['n_chars'].iloc[0]

# option 3 - First letter
first_letter = demo_wg_df['first_letter'].iloc[0]

# option 4 - Least common letter
least_common_letter = demo_wg_df['letter_selector'].iloc[0][0]

# option 5 - Multiple least common letters
letter_selector = demo_wg_df['letter_selector'].iloc[0]

# option 6 - Number of characters and multiple least common letters
nc_ls_tuple = demo_wg_df['nc_ls_tuple'].iloc[0]


## Select on the full matrix: option 1

In [15]:
# demo the full matrix selection
output = get_values_full_matrix(wg_id = wg_id, 
                    wchar_matrix = wchar_matrix,
                   word_group_id_list = word_group_id_list)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


## Select on the matrices split by word-length: option 2

In [16]:
# demo the n char selection
output = get_values_n_char(wg_id = wg_id,
                      n_char = n_char,
                      n_char_matrix_dict = n_char_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


## Select on the matrices split by the first letter: option 3

In [17]:
# demo the first letter selection
output = get_values_single_letter(wg_id = wg_id, single_letter = least_common_letter,
                                                           single_letter_matrix_dict = single_letter_matrix_dict)         

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


## Select on the matrices split by the single least common letter: option 4

In [18]:
# demo the first letter selection
output = get_values_single_letter(wg_id = wg_id, single_letter = least_common_letter,
                                                           single_letter_matrix_dict = single_letter_matrix_dict)         

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


## Select on the matrices split by the letter selector: option 5

In [19]:
# demo with the letter selector
output = get_values_letter_selector(wg_id = wg_id,
                      letter_selector = letter_selector,
                      letter_selector_matrix_dict = letter_selector_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


## Select on the matrices split by word-length and the letter selector: option 6

In [20]:
# demo with the n_char letter selector
output = get_values_n_char_letter_selector(wg_id = wg_id,
                           nc_ls_tuple = nc_ls_tuple,                           
                           nc_ls_matrix_dict=nc_ls_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


In [21]:
# we've tested with one word, let's time many evaluations to get a sense of how quickly 
# the different matrix extraction options work
# use the timeit() function to evaluate how long, on average, a single matrix operation
# takes to complete
# we do this by writing python code encapsulated in quotes which is then sent to the function
# we can store the quoted code in dictionary and then enumerate. 

n_trials = 10

code_snippet_dict = {
    'Matrix Selection Option 1: Selecting by full matrix':
"""get_values_full_matrix(wg_id = wg_id, wchar_matrix = wchar_matrix, word_group_id_list = word_group_id_list)""",
    'Matrix Selection Option 2: Selecting by word length':
"""get_values_n_char(wg_id = wg_id, n_char = n_char, n_char_matrix_dict = n_char_matrix_dict)""",
    'Matrix Selection Option 3: Selecting by first letter':
"""get_values_single_letter(wg_id = wg_id, single_letter = first_letter, single_letter_matrix_dict = single_letter_matrix_dict)""",    
    'Matrix Selection Option 4: Selecting by single least common letter':
"""get_values_single_letter(wg_id = wg_id, single_letter = least_common_letter, single_letter_matrix_dict = single_letter_matrix_dict)""",
    'Matrix Selection Option 5: Selecting by letter selector':
"""get_values_letter_selector(wg_id = wg_id, letter_selector = letter_selector, letter_selector_matrix_dict = letter_selector_matrix_dict)""",
    'Matrix Selection Option 6: Selecting by word length and letter selector':
"""get_values_n_char_letter_selector(wg_id = wg_id, nc_ls_tuple = nc_ls_tuple, nc_ls_matrix_dict=nc_ls_matrix_dict)"""
}

timing_list = []
for csd, cs in code_snippet_dict.items():    

    # here we time the code execution
    total_time = timeit.timeit(cs, number=n_trials, globals=globals())
    timing_list.append([csd, total_time])

    # total time    
    total_time_formatted = '{:,}'.format(round(total_time, 4))    

    # average time
    avg_time = total_time / n_trials
    avg_time_formatted = '{:,}'.format(round(avg_time, 6)) 
        
    print(csd)
    # average number of seconds per trial
    print('Total time:', total_time_formatted, 'seconds. Average time:', avg_time_formatted, 'seconds.')

    # add a line break to make it easier to read
    print()
    
    

Matrix Selection Option 1: Selecting by full matrix
Total time: 2.6046 seconds. Average time: 0.260464 seconds.

Matrix Selection Option 2: Selecting by word length
Total time: 1.9373 seconds. Average time: 0.193727 seconds.

Matrix Selection Option 3: Selecting by first letter
Total time: 1.9006 seconds. Average time: 0.190056 seconds.

Matrix Selection Option 4: Selecting by single least common letter
Total time: 0.1877 seconds. Average time: 0.018774 seconds.

Matrix Selection Option 5: Selecting by letter selector
Total time: 0.0016 seconds. Average time: 0.000163 seconds.

Matrix Selection Option 6: Selecting by word length and letter selector
Total time: 0.0018 seconds. Average time: 0.000177 seconds.



In [22]:
# the combination of the word length and letter selector is the fastest

In [23]:
# these numbers are orders of magnitude different and therefore hard to interpret.
# let's rescale.

In [34]:
# we'll use product() to compute the cartersian product of the list of timings
# from the timing list, we can compute the ratio of one timing to another
# we can then build a dataframe and cross-tab

expanded_timing_list = []
for me_source, me_target in product(timing_list, repeat=2):    
    # let's unpack this
    me_source_option, me_source_timing = me_source
    me_target_option, me_target_timing = me_target

    me_source_option = me_source_option[17:25]
    me_target_option = me_target_option[17:25]
    me_source_target_timing_ratio = me_source_timing / me_target_timing
    
    # add to the list
    expanded_timing_list.append([me_source_option, me_target_option, me_source_timing, me_target_timing, me_source_target_timing_ratio])



    

In [35]:
col_names = ['me_source', 'me_target', 'me_source_timing', 'me_target_timing', 'me_timing_ratio']

In [36]:
timing_df = pd.DataFrame(data = expanded_timing_list, columns = col_names)

In [37]:
# use the pd.pivot_table() function to display the ratios

In [49]:
# we computed the product so all cells will be filled
# but we only need to look at the top-diagonal
# from left to right: we can see how much faster option 5 is than option 1
# or how much faster option 4 is than option 3
timing_table = timing_df.pivot_table(index = ['me_source'], columns = ['me_target'],
                           values =['me_timing_ratio'])

In [48]:
timing_table

me_source me_timing_ratio                                 \
me_target                  Option 1  Option 2  Option 3   Option 4   
0          Option 1        1.000000  1.344492  1.370459  13.873856   
1          Option 2        0.743776  1.000000  1.019314  10.319036   
2          Option 3        0.729682  0.981052  1.000000  10.123510   
3          Option 4        0.072078  0.096908  0.098780   1.000000   
4          Option 5        0.000627  0.000843  0.000859   0.008698   
5          Option 6        0.000678  0.000911  0.000929   0.009405   

                                     
me_target     Option 5     Option 6  
0          1595.099204  1475.213810  
1          1186.395855  1097.228026  
2          1163.915973  1076.437699  
3           114.971584   106.330483  
4             1.000000     0.924841  
5             1.081266     1.000000